In [194]:
#實驗費、工讀費、研討會便當搜尋機
from PttWebCrawler.crawler import *
import re
import facebook
import requests
import json
import csv
import pandas as pd
import datetime
import time
import os
from lxml import etree
#from bs4 import BeautifulSoup
def filter_lt(predicate, lt):
    result = []
    for ele in lt:
        if predicate(ele):
            result.append(ele)
    return result
ptt_ntu_board_firstpage = "https://www.ptt.cc/bbs/NTU/index.html"
goods_keyword = '(.*便當|.*受試者|.*車馬費|.*禮券|.*贈送)'
diff_time_to_now_in_secs_standard = 60*60*24

In [76]:
r = requests.get(ptt_ntu_board_firstpage)
html = etree.HTML(r.text)
result = html.xpath('//*[@id="action-bar-container"]/div/div[2]/a[2]/@href')
newestpagei = int(re.sub(pattern="[a-zA-Z/.]+",repl="",string=result[0]))+1
c = PttWebCrawler(as_lib=True)
crawlpages = 5
c.parse_articles(newestpagei-crawlpages, newestpagei, 'NTU')
path_to_ptt_ntu_json_file = 'NTU-'+str(newestpagei-crawlpages)+"-"+str(newestpagei)+".json"

Processing index: 2553
Processing article: M.1555127252.A.2AD
Processing article: M.1555133931.A.99D
Processing article: M.1555144733.A.05A
Processing article: M.1555158533.A.046
Processing article: M.1555169896.A.AD6
Processing index: 2554
Processing article: M.1555179499.A.C7A
Processing article: M.1555208118.A.4C5
Processing index: 2555
Processing index: 2556
Processing article: M.1555259399.A.7AE
Processing article: M.1555293983.A.FFB
Processing article: M.1555301768.A.B1A
Processing article: M.1555306791.A.24E
Processing article: M.1555307324.A.921
Processing article: M.1555308932.A.936
Processing article: M.1555313665.A.FED
Processing article: M.1555314485.A.3DA
Processing article: M.1555318020.A.7F4
Processing article: M.1555319433.A.5AF
Processing index: 2557
Processing article: M.1555325571.A.EAF
Processing article: M.1555382958.A.1C1
Processing article: M.1555395081.A.DDF
Processing article: M.1555404324.A.C17
Processing article: M.1555409976.A.00A
Processing article: M.15554

In [180]:
print(newestpagei)
print(path_to_ptt_ntu_json_file)
with open(path_to_ptt_ntu_json_file , 'r') as reader:
    freemeal = json.loads(reader.read())
    freemeal = freemeal["articles"]
#os.remove(path_to_ptt_ntu_json_file)

for jf_item_i in range(0,len(freemeal)) :
    del freemeal[jf_item_i]['messages']
    del freemeal[jf_item_i]['message_count']

freemeal = pd.DataFrame.from_dict(freemeal)
freemeal = freemeal[freemeal["date"]!=""]

def convert_ptt_date_time(str):
    lt = str.split(" ")
    weekd, month, monthd, time, year = (filter_lt(lambda ele: ele!="", lt))
    monthd = (monthd.zfill(2))
    newdt = weekd+"-"+month+"-"+monthd+"-"+time+"-"+year
    dt_object = datetime.datetime.strptime(newdt, '%a-%b-%d-%H:%M:%S-%Y')
    return(dt_object)
    #Thu Apr 18 20:54:40 2019
    #en_US:%a %b %d %H:%M%S %Y
freemeal = freemeal.assign(date_time=freemeal.loc[:,"date"].apply(convert_ptt_date_time))
freemeal = freemeal[freemeal.article_title.str.match(goods_keyword) | freemeal.content.str.match('(.*便當|.*受試者|.*車馬費|.*禮券|.*贈送)')]
freemeal = freemeal.assign(diffsec_to_now=freemeal.loc[:,"date_time"].apply(lambda diff: (datetime.datetime.now()-diff).total_seconds()))

2558
NTU-2553-2558.json


In [199]:
timelyfreemeal = freemeal[freemeal.diffsec_to_now<diff_time_to_now_in_secs_standard]
timelyfreemeal = timelyfreemeal.assign(fullcontent = timelyfreemeal.apply(lambda x: ''.join(str(x)), axis=1))
if (timelyfreemeal.shape[0]==0):
    print("there is no free meal")
    sys.exit()

print("there are free meals")
print(timelyfreemeal)
goods_message = ("\n============\n".join(timelyfreemeal.fullcontent))

import smtplib
from email.mime.text import MIMEText
from email.header import Header
server = smtplib.SMTP('smtps.ntu.edu.tw', 465)
server.login("youremailusername", "password")
goods_message = ""
message = MIMEText('免費好康通知'+goods_message, 'plain', 'utf-8')
message['From'] = Header("通知人@ntu.edu.tw", 'utf-8')
message['To'] =  Header("受通報人@ntu.edu.tw", 'utf-8')
 
subject = '來自台大的好康通報！'
message['Subject'] = Header(subject, 'utf-8')
server.sendmail(message['From'], message['To'], message.as_string())

there are free meals
            article_id       article_title            author board  \
33  M.1555558464.A.CBD  [徵求] 台大腦心所徵求MRI受試者  Myscty (pumpkin)   NTU   

                                              content  \
33  [email protected] (請勿寄站內信，謝謝) （實驗室電話：022312345...   

                        date               ip  \
33  Thu Apr 18 11:34:22 2019  140.112.122.158   

                                                  url           date_time  \
33  https://www.ptt.cc/bbs/NTU/M.1555558464.A.CBD.... 2019-04-18 11:34:22   

    diffsec_to_now                                        fullcontent  
33    50853.405392  article_id                                    ...  
